In [1]:
#Import libraries
import sys
import pymol
from pymol import cmd

In [ ]:
# Run PyMOL
pymol.finish_launching()

In [3]:
# Selects all residues in the specified protein
def select_all_residues(protein_id):
    cmd.select("all_residues", protein_id)
    #print("All residues in the protein have been selected.")

In [4]:
def select_residues_within_distance(interactor, angstrom, output_file):

    # Define a reference selection if it's not already defined
    if not cmd.count_atoms(interactor):
        #print(f"Error: Selection '{interactor}' not found.")
        return
    
    # Select residues within the specified distance around the reference selection
    cmd.select("residues_within_6A", f"byres {interactor} around {angstrom}")
    
    # Get the model of the newly selected residues
    selected_residues = cmd.get_model("residues_within_6A")
    
    # Keep track of unique residue IDs
    unique_residues = set()
    
    # Write the amino acids and positions to the output file
    with open(output_file, "w") as f:
        for atom in selected_residues.atom:
            if atom.resn and atom.resi:
                residue_id = (atom.resn, atom.resi)
                if residue_id not in unique_residues:
                    f.write(f"{atom.resn} {atom.resi}\n")
                    unique_residues.add(residue_id)
    
    #print(f"Residues within {angstrom} Ångstroms of the reference selection have been written to {output_file}.")


In [24]:
def fetch_protein(protein_id):
    # Fetch the protein into a temporary object
    cmd.fetch(protein_id, type="pdb", async_=0)
    cmd.remove("solvent")
    
    # Generate a unique object name
    temp_object_name = f"{protein_id}_temp"
    if temp_object_name in cmd.get_object_list():
        temp_object_name += "_1"  # Append a suffix if the name is already in use
    
    # Debug output
    print(f"Creating object '{temp_object_name}' with selection '{protein_id}'")
    
    # Append the temporary object to the existing object
    cmd.create(protein_id, temp_object_name)
    cmd.delete(temp_object_name)

In [19]:
# Define the protein IDs
protein1_id = "3lhr"
protein2_id = "SCAN_domain_P28698_model0_checked_Repair"

In [20]:
# Construct the output file name
output_file = f"complex_residues_with_{protein1_id}.txt"

In [ ]:
# Fetch the proteins and select residues
fetch_protein(protein1_id)

In [ ]:
# Load the PDB file for protein2_id
cmd.load("../data/SCAN_domain_P28698_model0_checked_Repair.pdb", protein2_id)

In [ ]:
select_all_residues(protein1_id)

In [ ]:
select_residues_within_distance("all_residues", 6, output_file)

In [12]:
cmd.remove("all")